In [1]:
import requests
import json
import math
from bs4 import BeautifulSoup as bs
import re
import numpy as np
import pandas as pd

In [2]:
from flask_sqlalchemy import *#SQLAlchemy
from sqlalchemy import create_engine
import sqlite3

# # #Connecting to sqlite
# conn = sqlite3.connect('zip_data.sqlite')

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Doping EMPLOYEE table if already exists
# cursor.execute("DROP TABLE area")
# print("Table dropped... ")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()

In [37]:
con = create_engine("sqlite:///zip_data2.sqlite")

In [38]:
df = pd.read_sql("area",con)

In [28]:
# zips = []
# for n in range(1001,99950+1):
#     n = str(n).zfill(5)
#     zips.append(n)
# print(len(zips))


In [34]:
con0 = create_engine("sqlite:///us_db.sqlite")
ref = pd.read_sql("city",con0)
ref = ref[ref["zip"].astype(int)>=1000]
ref = list(ref["zip"])
ref = pd.DataFrame({"zip":ref})

#to get zip codes already in database
exist = pd.read_sql("area",con)
exist = list(exist["zip"])
#zips already in database will have exist value of 1, if not NaN
exist = pd.DataFrame({"zip":exist,"exist":1})

#merge on reference dataframe
zips_srch = pd.merge(ref,exist,on="zip",how="left")
#excluding rows where exist = 1
zips_srch = zips_srch[zips_srch["exist"].isnull()]
zips = list(zips_srch["zip"])

In [10]:
# try:
#     area = pd.read_sql("area",con)
#     zip_exist = list(area["zip"])

#     # excluding zipcodes that are already in the database
#     # zips_o = [x for x in zips if x not in zip_exist]
#     zips_df = pd.DataFrame({"zips":zips})
#     exist_df = pd.DataFrame({"zips":zip_exist,"exist":1})
#     zips_df = pd.merge(zips_df,exist_df, how="left",on = "zips")
#     # exist_df
#     zips = list(zips_df[zips_df["exist"].isnull()]["zips"])
#     print(len(zips))
# except OperationalError:
#     print("none")

25412


In [11]:
for z in zips:
    url = f"http://www.mapszipcode.com/state/city/{z}/"
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    # divs = soup.find_all("div", {"class": "dat"})
    divs = soup.find_all("div", class_="dat")
    try:
        population = int(str(divs[0]).split(">")[1].split("<")[0].replace(",",""))
    except ValueError:
        population = np.nan
    try:
        density = float(str(divs[1]).split(">")[1].split("	")[0].replace(",",""))
    except ValueError:
        density = np.nan
    try:
        med_home_value = float(str(divs[2]).split(">")[1].split("<")[0].replace(",","").replace("$",""))
    except ValueError:
        med_home_value = np.nan
    try:
        med_rent = float(str(divs[3]).split(">")[1].split("<")[0].replace(",","").replace("$",""))
    except ValueError:
        med_rent = np.nan
    # area = population/density
    # radius = math.sqrt(area)/2*1.6
    apnd = pd.DataFrame({"zip":[z],"population":[population],"density":[density],"med_home_value":[med_home_value],"med_rent":[med_rent]})
    apnd.to_sql("area",con,if_exists="append",index=False)

In [39]:
df = pd.read_sql("area",con)

In [41]:
df.dropna(inplace=True)

In [43]:
df.to_sql("area2",con0,if_exists="replace",index=False)

In [44]:
df1 = pd.read_sql("area",con0)

In [45]:
df2 = pd.read_sql("area2",con0)